# Introduction

Critical in evaluation of the NHM model simulated flows is comparison to observed flows in the watershed. This notebook retrieves available streamflow observations from NWIS and two state agencies, the Oregon Water Resources Deparment (OWRD) and the Washington Departent of Ecology (ECY), combines these data sets into one streamflow obervations file and streanflow gage information and metadata, and write the database out as a netCDF file (.nc) to be used in Notebook "6_Streamflow_Output_Visualization" and other notebooks in NHM-Assist. A complete database of streamflow gages and observation in the model domain is nessessary to evaluate the NHM model and identify other gages that could be included in a model recalibration to improve the model performance.

Three key facts about streamflow observations and the NHM must be reviewed.
- Streamflow observations are NOT used when running PRMS or pywatershed. These data are meant for comparison of simulated output only.
- The NHM DOES use streamflow observations from NWIS in the model calibration workflow (not the streamflow file).
- Limited streamflow gage information is stored in the parameter file.

The paramter file has  are dimensioned by npoigages and include :
- poi_gage_id, the agency identification number
- poi_gage_segment, model segment identification number (nhm_seg) on which the gage falls (1 gage/segment only),
- poi_type, historically used, but not currently used.

It is important to note that the gages in the parameter file are NOT a complete set of gages in the model domain, and NOT all used to calibrate the model. 

In [ ]:
# %run "0a_Workspace_setup.ipynb"
%run "0b_Create_poi_files.ipynb"

from NHM_helpers.sf_data_retrieval import *

# Get daily streamflow data for gages
##### We will not use the list of gages in the model paramter file, but will use the gages lsited in the gages_file.csv. This The reasoning: there may be multiple observation datasets that are associated with a single segment outflow (gage_poi), and, in the parameter file, only one poi_gage can be associated with a segment. We want a streamflow data set that is more inclusive.

## Retrieve availabale daily streamflow data from Oregon Water Resources Department
#### https://apps.wrd.state.or.us/apps/sw/hydro_near_real_time/

In [ ]:
owrd_regions = ["16", "17", "18"]

if any(item in owrd_regions for item in model_domain_regions):
    owrd_domain_txt = f"; The model domain intersects the Oregon state boundary, and streamflow data from OWRD will be retrieved."
    if output_netcdf_filename.exists():
        pass
    else:
        owrd_df = create_OR_sf_df(control, model_dir, gages_df)

else:
    owrd_domain_txt = f"; The model domain is outside the Oregon state boundary."

con.print(owrd_domain_txt)

## Retrieve availabale daily streamflow data from Washing Department of Ecology
#### https://waecy.maps.arcgis.com/apps/Viewer/index.html?appid=832e254169e640fba6e117780e137e7b

In [ ]:
ecy_regions = ["17"]

if any(item in ecy_regions for item in model_domain_regions):
    # Ecology gages in the gage list have a gage ID with 6 alpha/numeric characters: 2 numeric - one alpha - 2 numeric.

    if output_netcdf_filename.exists():
        ecy_domain_txt = f"The model domain intersects the Washington state boundary, and streamflow data from ECY was retrieved."
        pass
    else:
        ecy_df = create_ecy_sf_df(control, model_dir, gages_df)
        ecy_domain_txt = f"The model domain intersects the Washington state boundary, and streamflow data for {len(ecy_df)}from ECY was retrieved."
else:
    ecy_domain_txt = "The model domain is outside the Washington state boundary."

con.print(ecy_domain_txt)

Note: ecy_ds is going outside of the date range. use the time slice using ecy_start_date and ecy_end_date

# Retrieve availabale daily streamflow data from NWIS
#### https://waterdata.usgs.gov/nwis/sw

In [ ]:
if output_netcdf_filename.exists():
    pass
else:
    NWIS_df = create_nwis_sf_df(control, model_dir, gages_df)

## Combine daily streamflow dataframes (if needed)
note: all NWIS data is mirrored the OWRD database without any primary source tags. This section will also etermine the original source or each daily observation and create a tag for each daily record.

In [ ]:
if output_netcdf_filename.exists():
    pass
else:
    streamflow_df = NWIS_df.copy()  # Sets streamflow file to default, NWIS_df

    if (
        not owrd_df.empty
    ):  # If there is an owrd_df, it will be combined with streamflow_df and rewrite the streamflow_df
        # Merge NWIS and OWRD
        streamflow_df = pd.concat([streamflow_df, owrd_df])  # Join the two datasets
        # Drop duplicated indexes, keeping the first occurence (USGS occurs first)
        # try following this thing: https://saturncloud.io/blog/how-to-drop-duplicated-index-in-a-pandas-dataframe-a-complete-guide/#:~:text=Pandas%20provides%20the%20drop_duplicates(),names%20to%20the%20subset%20parameter.
        streamflow_df = streamflow_df[~streamflow_df.index.duplicated(keep="first")]

    elif (
        not ecy_df.empty
    ):  # If there is an ecy_df, it will be combined with streamflow_df and rewrite the streamflow_df
        streamflow_df = pd.concat([streamflow_df, ecy_df])
        streamflow_df = streamflow_df[~streamflow_df.index.duplicated(keep="last")]

    else:
        pass

#### Poteniatlly important code later for checking stuff

## Create Xarray dataset with streamflow date set (indexed by poi_id and time)

## Make and Xarray data set for NWIS, OWRD, and WA Ecology data and encode with station information

In [ ]:
if output_netcdf_filename.exists():
    pass
else:
    xr_station_info = xr.Dataset.from_dataframe(
        gages_df
    )  # gages_df is the new source of gage metadata
    xr_streamflow_only = xr.Dataset.from_dataframe(streamflow_df)
    xr_streamflow = xr.merge(
        [xr_streamflow_only, xr_station_info], combine_attrs="drop_conflicts"
    )
    # test_poi = xr_streamflow.poi_id.values[2]

    # xr_streamflow.agency_id.sel(poi_id=test_poi).to_dataframe().agency_id.unique()
    xr_streamflow = xr_streamflow.sortby("time", ascending=True)  # bug fix for xarray

In [ ]:
if output_netcdf_filename.exists():
    pass
else:
    # Set attributes for the variables
    xr_streamflow["discharge"].attrs = {"units": "ft3 s-1", "long_name": "discharge"}
    xr_streamflow["drainage_area"].attrs = {
        "units": "mi2",
        "long_name": "Drainage Area",
    }
    xr_streamflow["drainage_area_contrib"].attrs = {
        "units": "mi2",
        "long_name": "Effective drainage area",
    }
    xr_streamflow["latitude"].attrs = {
        "units": "degrees_north",
        "long_name": "Latitude",
    }
    xr_streamflow["longitude"].attrs = {
        "units": "degrees_east",
        "long_name": "Longitude",
    }
    xr_streamflow["poi_id"].attrs = {
        "role": "timeseries_id",
        "long_name": "Point-of-Interest ID",
        "_Encoding": "ascii",
    }
    xr_streamflow["poi_name"].attrs = {
        "long_name": "Name of POI station",
        "_Encoding": "ascii",
    }
    xr_streamflow["time"].attrs = {"standard_name": "time"}
    xr_streamflow["poi_agency"].attrs = {"_Encoding": "ascii"}
    xr_streamflow["agency_id"].attrs = {"_Encoding": "ascii"}

    # Set encoding
    # See 'String Encoding' section at https://crusaderky-xarray.readthedocs.io/en/latest/io.html
    xr_streamflow["poi_id"].encoding.update(
        {"dtype": "S15", "char_dim_name": "poiid_nchars"}
    )

    xr_streamflow["time"].encoding.update(
        {
            "_FillValue": None,
            "calendar": "standard",
            "units": "days since 1940-01-01 00:00:00",
        }
    )

    xr_streamflow["latitude"].encoding.update({"_FillValue": None})
    xr_streamflow["longitude"].encoding.update({"_FillValue": None})

    xr_streamflow["agency_id"].encoding.update(
        {"dtype": "S5", "char_dim_name": "agency_nchars"}
    )

    xr_streamflow["poi_name"].encoding.update(
        {"dtype": "S50", "char_dim_name": "poiname_nchars"}
    )

    xr_streamflow["poi_agency"].encoding.update(
        {"dtype": "S5", "char_dim_name": "mro_nchars", "_FillValue": ""}
    )
    # Add fill values to the data variables
    var_encoding = dict(_FillValue=netCDF4.default_fillvals.get("f4"))

    for cvar in xr_streamflow.data_vars:
        if xr_streamflow[cvar].dtype != object and cvar not in [
            "latitude",
            "longitude",
        ]:
            xr_streamflow[cvar].encoding.update(var_encoding)

    # add global attribute metadata
    xr_streamflow.attrs = {
        "Description": "Streamflow data for PRMS",
        "FeatureType": "timeSeries",
    }

## Assign EFC values to the Xarray dataset

In [ ]:
# Attributes for the EFC-related variables
attributes = {
    "efc": {
        "dtype": np.int32,
        "attrs": {
            "long_name": "Extreme flood classification",
            "_FillValue": -1,
            "valid_range": [1, 5],
            "flag_values": [1, 2, 3, 4, 5],
            "flag_meanings": "large_flood small_flood high_flow_pulse low_flow extreme_low_flow",
        },
    },
    "ri": {
        "dtype": np.float32,
        "attrs": {
            "long_name": "Recurrence interval",
            "_FillValue": 9.96921e36,
            "units": "year",
        },
    },
    "high_low": {
        "dtype": np.int32,
        "attrs": {
            "long_name": "Discharge classification",
            "_FillValue": -1,
            "valid_range": [1, 3],
            "flag_values": [1, 2, 3],
            "flag_meanings": "low_flow ascending_limb descending_limb",
        },
    },
}

In [ ]:
if output_netcdf_filename.exists():
    with xr.open_dataset(output_netcdf_filename) as model_output:
        xr_streamflow = model_output
        del model_output
else:
    var_enc = {}
    for var, info in attributes.items():
        # Add the variable
        xr_streamflow[var] = xr.zeros_like(
            xr_streamflow["discharge"], dtype=info["dtype"]
        )

        var_enc[var] = {"zlib": True, "complevel": 2}

        # Take care of the attributes
        del xr_streamflow[var].attrs["units"]

        for kk, vv in info["attrs"].items():
            if kk == "_FillValue":
                var_enc[var][kk] = vv
            else:
                xr_streamflow[var].attrs[kk] = vv

In [ ]:
%%time
if output_netcdf_filename.exists():
    pass
else:

    flow_col = "discharge"

    for pp in xr_streamflow.poi_id.data:
        try:
            df = efc(
                xr_streamflow.discharge.sel(poi_id=pp).to_dataframe(), flow_col=flow_col
            )

            # Add EFC values to the xarray dataset for the poi
            xr_streamflow["efc"].sel(poi_id=pp).data[:] = df.efc.values
            xr_streamflow["high_low"].sel(poi_id=pp).data[:] = df.high_low.values
            xr_streamflow["ri"].sel(poi_id=pp).data[:] = df.ri.values
        except TypeError:
            pass

## Write the Xarray data set to a netcdf file

In [ ]:
if output_netcdf_filename.exists():
    con.print(
        f"The output file already exists. [bold]To re-write[/] the file, please [bold]remove[/] the existing file:"
    )
    con.print(f"[bright_magenta]{output_netcdf_filename}[/]")
else:
    xr_streamflow.to_netcdf(output_netcdf_filename)

### Check file: plot individual POI with EFC highlighted

In [ ]:
cpoi_id = xr_streamflow.poi_id.values[0]

ds_sub = xr_streamflow.sel(poi_id=cpoi_id, time=slice("1980-10-01", "2022-12-31"))
ds_sub = ds_sub.to_dataframe()
flow_col = "discharge"

In [ ]:
plot_efc(ds_sub, flow_col)

In [ ]:
plot_high_low(ds_sub, flow_col)